In [1]:
addprocs(5);

In [5]:
using DistributedArrays

In [2]:
nprocs()

6

In [6]:
@everywhere function par(I)
  d=(length(I[1]),length(I[2]))
  m=fill(myid(),d)
  println(I)
  #println(d)
  return m
end

In [7]:
m=DArray(par,(8,8),[2:5])

 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at /Applications/Julia-0.4.0.app/Contents/Resources/julia/lib/julia/sys.dylib
 in vect at abstractarray.jl:32
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /Users/ppalmes/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /Users/ppalmes/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading In[7], in expression starting on line 1


	From worker 2:	(1:4,1:4)
	From worker 5:	(5:8,5:8)
	From worker 3:	(5:8,1:4)
	From worker 4:	(1:4,5:8)


8x8 DistributedArrays.DArray{Int64,2,Array{Int64,2}}:
 2  2  2  2  4  4  4  4
 2  2  2  2  4  4  4  4
 2  2  2  2  4  4  4  4
 2  2  2  2  4  4  4  4
 3  3  3  3  5  5  5  5
 3  3  3  3  5  5  5  5
 3  3  3  3  5  5  5  5
 3  3  3  3  5  5  5  5

In [8]:
m.indexes

2x2 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:4,1:4)  (1:4,5:8)
 (5:8,1:4)  (5:8,5:8)

In [9]:
m.chunks

2x2 Array{RemoteRef{RemoteStore},2}:
 RemoteRef{Channel{Any}}(2,1,26)  RemoteRef{Channel{Any}}(4,1,28)
 RemoteRef{Channel{Any}}(3,1,27)  RemoteRef{Channel{Any}}(5,1,29)

In [10]:
fetch(@spawnat 2 localpart(m))

4x4 Array{Int64,2}:
 2  2  2  2
 2  2  2  2
 2  2  2  2
 2  2  2  2

In [11]:
procs(m)

2x2 Array{Int64,2}:
 2  4
 3  5

In [14]:
map(fetch,[@spawnat p sum(localpart(m)) for p in procs(m)])

4-element Array{Any,1}:
 32
 48
 64
 80

In [15]:
reduce(+,map(fetch,[@spawnat p sum(localpart(m)) for p in procs(m)]))

224

In [10]:
# "Map" function.
# Takes a string. Returns a HashTable with the number of times each word 
# appears in that string.
@everywhere function wordcount(text)
    words=split(text,r"\s+|'\n'|'\t'|:|;|,|!|\"|\.|,",false)
    counts=Dict()
    for w in words
        counts[w]=get(counts,w,0)+1
    end
    return counts
end

In [12]:
# "Reduce" function.
# Takes a collection of HashTables in the format returned by wordcount()
# Returns a HashTable in which words that appear in multiple inputs
# have their totals added together.
@everywhere function wcreduce(wcs)
  counts=Dict()
  for words_dict in wcs
    for (k,v) in words_dict
            counts[k] = get(counts,k,0)+v
        end
    end
    return counts
end


In [ ]:
## Splits input string into nprocs() equal-sized chunks (last one rounds up), 
## and @spawns wordcount() for each chunk to run in parallel. Then fetch()s
## results and performs wcreduce().
## Limitations: splitting the string and reduction step are single-threaded.
#@everywhere function parallel_wordcount(text)
#    lines=split(text,r"'\n'|-|/|@|<|>",false)
#    np=nprocs()
#    unitsize=ceil(length(lines)/np)
#    wcounts={}
#    rrefs={}
#    # spawn procs
#    for i=1:np
#        first=unitsize*(i-1)+1
#        last=unitsize*i
#        if last>length(lines)
#            last=length(lines)
#        end
#        subtext=join(lines[int(first):int(last)],"\n")
#        push!(rrefs, @spawn wordcount( subtext ) )
#    end
#    # fetch results
#    while length(rrefs)>0
#        push!(wcounts,fetch(pop!(rrefs)))
#    end
#    # reduce
#    count=wcreduce(wcounts)
#    return count
#end

In [13]:
# Splits input string into nprocs() equal-sized chunks (last one rounds up), 
# and @spawns wordcount() for each chunk to run in parallel. Then fetch()s
# results and performs wcreduce().
# Limitations: splitting the string and reduction step are single-threaded.
@everywhere function parallel_wordcount(text)
    lines=split(text,r"'\n'|-|/|@|<|>",false)
    np=nprocs()
    unitsize=ceil(length(lines)/np)
    wcounts={}
    rrefs={}
    # spawn procs
    wcounts=@parallel (hcat) for i=1:np
        first=unitsize*(i-1)+1
        last=unitsize*i
        if last>length(lines)
            last=length(lines)
        end
        subtext=join(lines[int(first):int(last)],"\n")
        wordcount( subtext )
    end
    # reduce
    count=wcreduce([wcounts])
    return count
end

In [ ]:
## Takes the name of a result file, and a list of input file names.
## Combines the contents of all files, then performs a parallel_wordcount
## on the resulting string. Writes the results to result_file.
## Limitation: Performs all file IO single-threaded.
#function wordcount_files(input_file_names)
#    text=""
#    for f = input_file_names
#        fh=open(f)
#        text=join( {text,readall(fh)}, "\n" )
#        close(fh)
#    end
#    wc=parallel_wordcount(text)
#    for (k,v) = wc
#        println(k,"=",v)
#    end
#end

In [14]:
# Takes the name of a result file, and a list of input file names.
# Combines the contents of all files, then performs a parallel_wordcount
# on the resulting string. Writes the results to result_file.
# Limitation: Performs all file IO single-threaded.
@everywhere function wordcount_files(input_file_names)
  alltext=@parallel (hcat) for f in input_file_names
        fh=open(f)
        text=readall(fh)
        close(fh)
        text
    end
    if length(input_file_names)>1 
        alltext=join(alltext," ") #to string
    end
    wc=parallel_wordcount(alltext)
    for (k,v) = wc
        println(k,"=",v)
    end
end

In [15]:
args=["input1.txt","input2.txt","input2.txt"]#"input3.txt"]
wordcount_files(args)

hello=9
worlds=1
world=8


In [16]:
a=rand(1000,1000)

1000x1000 Array{Float64,2}:
 0.342043   0.136707   0.790181    …  0.226505  0.737679   0.00803246
 0.690561   0.945658   0.750886       0.742464  0.738643   0.941033  
 0.737747   0.749402   0.112646       0.470658  0.609533   0.734104  
 0.759085   0.973392   0.0503567      0.299679  0.0966368  0.0295505 
 0.0555818  0.791415   0.0486997      0.787038  0.259832   0.360027  
 0.56686    0.895342   0.946203    …  0.474043  0.305913   0.0804972 
 0.907674   0.298624   0.420697       0.884067  0.18983    0.405377  
 0.104182   0.476202   0.108984       0.290297  0.222477   0.0193876 
 0.497331   0.0495303  0.205866       0.487488  0.0610598  0.110208  
 0.391265   0.724377   0.297801       0.350274  0.220217   0.311541  
 0.456254   0.990781   0.423237    …  0.826567  0.356826   0.477251  
 0.257105   0.236355   0.264739       0.682232  0.826635   0.428883  
 0.536725   0.76173    0.256634       0.199764  0.0374949  0.27582   
 ⋮                                 ⋱                          

In [17]:
d=distribute(a)

1000x1000 DistributedArrays.DArray{Float64,2,Array{Float64,2}}:
 0.342043   0.136707   0.790181    …  0.226505  0.737679   0.00803246
 0.690561   0.945658   0.750886       0.742464  0.738643   0.941033  
 0.737747   0.749402   0.112646       0.470658  0.609533   0.734104  
 0.759085   0.973392   0.0503567      0.299679  0.0966368  0.0295505 
 0.0555818  0.791415   0.0486997      0.787038  0.259832   0.360027  
 0.56686    0.895342   0.946203    …  0.474043  0.305913   0.0804972 
 0.907674   0.298624   0.420697       0.884067  0.18983    0.405377  
 0.104182   0.476202   0.108984       0.290297  0.222477   0.0193876 
 0.497331   0.0495303  0.205866       0.487488  0.0610598  0.110208  
 0.391265   0.724377   0.297801       0.350274  0.220217   0.311541  
 0.456254   0.990781   0.423237    …  0.826567  0.356826   0.477251  
 0.257105   0.236355   0.264739       0.682232  0.826635   0.428883  
 0.536725   0.76173    0.256634       0.199764  0.0374949  0.27582   
 ⋮                        

In [20]:
@time reduce(+,map(fetch,[@spawnat p sum(localpart(d)) for p in procs(d)]))

500072.1977973655

0.007764 seconds (4.43 k allocations: 342.813 KB)


In [19]:
@time reduce(+,d)

500072.1977973655

0.095651 seconds (22.72 k allocations: 1.229 MB)
